In [1]:
# Import the necessary packages

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

import re
import emoji

from sklearn.feature_extraction.text import CountVectorizer #for bag of words

import gensim
from gensim.models import Word2Vec

from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy import interpolate
from scipy.spatial import ConvexHull

from sklearn.manifold import SpectralEmbedding

from sklearn.manifold import TSNE

from sklearn.linear_model import LogisticRegressionCV

import pandas as pd
import numpy as np

import os
path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Kaggle_istanbul/'
os.chdir(path)

import random
random.seed(465) #seed

## Read the data

In [2]:
# Training data set
X_train = pd.read_csv('X_train_bow.csv')
X_train.drop(columns=X_train.columns[0], axis=1, inplace=True)
X_train

,favorite_count,retweet_count,year,bow_pc1,bow_pc2,bow_pc3,bow_spec1,bow_spec2,bow_spec3,bow_tsne1,...,bow_tsne3,hash_pc1,hash_pc2,hash_pc3,hash_spec1,hash_spec2,hash_spec3,hash_tsne1,hash_tsne2,hash_tsne3
0,0,10,2017.0,0.069350,-0.032713,-0.330951,0.009920,-0.003727,-0.001900,-56.124020,...,133.327347,-0.136923,2.022559e-17,7.329820e-18,-0.010656,-0.002951,0.004534,175.057037,91.138161,9.511187
1,258,111,2020.0,-1.307896,0.772888,-5.427592,-0.007626,0.038863,-0.025159,-55.414528,...,-111.317375,-0.136923,1.066861e-15,1.852020e-18,-0.010656,-0.002951,0.004534,14.681177,-153.787506,94.273918
2,0,2,2014.0,-0.303450,-0.687232,-0.205290,0.009920,-0.003727,-0.001900,135.679062,...,133.643921,-0.136923,-6.217318e-16,-2.370566e-16,0.033321,-0.006671,0.006562,20.365318,-132.089554,-62.750347
3,9,3,2017.0,-0.616210,0.006571,0.135430,-0.015783,-0.010377,-0.004557,-83.961861,...,-110.023521,-0.182829,-3.998495e-15,2.336553e-15,-0.010656,-0.002951,0.004534,137.812347,-132.927887,16.813818
4,3,3,2019.0,-1.831387,-0.472631,0.145726,-0.015783,-0.010377,-0.004557,119.542198,...,140.797729,-0.136923,-1.610217e-15,4.970175e-15,0.033321,-0.006671,0.006562,36.458218,54.029251,180.451508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1720,176,2019.0,-0.372858,0.047064,-0.083379,0.000032,0.000104,0.000012,83.362183,...,35.381844,-0.136923,-1.206374e-16,1.686205e-15,-0.010656,-0.002951,0.004534,-115.964424,-97.748260,-120.740288
96,1,2,2019.0,-0.054620,-2.935883,0.285268,-0.015783,-0.010377,-0.004557,-77.853325,...,106.027527,-0.182829,4.009226e-15,-2.288459e-15,-0.010656,-0.002951,0.004534,-150.312927,-1.687960,60.999504
97,28,8,2018.0,0.206137,-3.583367,-2.958666,-0.007626,0.038863,-0.025159,-156.558701,...,48.756199,-0.181126,3.467071e-15,-1.272427e-15,-0.010656,-0.002951,0.004534,165.070908,-105.786003,-43.867970
98,5,2,2017.0,-1.798368,-6.247552,-16.375284,0.003918,-0.013762,-0.000865,1.413357,...,189.845428,-0.136923,1.165245e-16,4.709362e-16,0.033321,-0.006671,0.006562,-23.753330,28.287981,-15.287249


In [3]:
# Training data set
y_train = pd.read_csv('y_train_bow.csv')
y_train.drop(columns=y_train.columns[0], axis=1, inplace=True)
y_train

,party_class
0,1
1,1
2,1
3,1
4,0
...,...
95,0
96,0
97,1
98,0


In [4]:
# Test data set
X_test = pd.read_csv('X_test_bow.csv')
X_test.drop(columns=X_test.columns[0], axis=1, inplace=True)
X_test

,favorite_count,retweet_count,year,bow_pc1,bow_pc2,bow_pc3,bow_spec1,bow_spec2,bow_spec3,bow_tsne1,...,bow_tsne3,hash_pc1,hash_pc2,hash_pc3,hash_spec1,hash_spec2,hash_spec3,hash_tsne1,hash_tsne2,hash_tsne3
0,70,13,2018.0,-0.588585,-0.614404,-0.400665,-0.000920,-0.004015,-0.002287,-181.846725,...,93.218765,-0.144775,-4.792006e-16,1.605697e-15,0.001094,0.000659,-0.000685,7.627090,-5.036632,8.366401
1,49,24,2017.0,-0.674279,-2.814501,-3.741224,-0.015783,-0.010377,-0.004557,74.847282,...,143.860428,-0.182829,-3.193260e-16,5.291457e-16,0.033321,-0.006671,0.006562,-175.951401,-66.239883,-36.245705
2,14,2,2019.0,2.739424,-3.024594,-0.147748,-0.015783,-0.010377,-0.004557,43.164875,...,135.554382,-0.136923,5.028518e-16,5.074727e-16,-0.010656,-0.002951,0.004534,-50.927887,61.181335,34.913677
3,13,7,2018.0,-0.447878,-0.875320,0.225986,-0.015783,-0.010377,-0.004557,94.442009,...,-31.132038,-0.182829,1.941753e-15,7.109076e-16,-0.010656,-0.002951,0.004534,-62.385738,-109.868660,-60.549969
4,73,13,2017.0,-0.434590,-0.969817,-0.482866,0.011693,-0.003208,0.001804,102.979500,...,-17.966028,-0.136923,-1.256897e-16,6.868628e-16,-0.010656,-0.002951,0.004534,-811.860046,76.306595,4.394346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2,2015.0,-0.550316,-0.398828,0.100865,-0.015783,-0.010377,-0.004557,94.402832,...,-1.413965,-0.136923,4.263764e-15,7.233052e-16,-0.010656,-0.002951,0.004534,132.282791,44.335316,-154.416290
96,7,6,2017.0,-0.431753,-0.465304,-0.052556,-0.015783,-0.010377,-0.004557,20.344114,...,-90.821556,-0.136923,1.668177e-15,4.478139e-16,-0.010656,-0.002951,0.004534,25.384073,75.764824,-47.454273
97,7,3,2020.0,-1.192181,-2.219881,-0.390333,0.009920,-0.003727,-0.001900,-129.740219,...,-58.800472,-0.136923,5.401520e-17,1.531979e-16,-0.010656,-0.002951,0.004534,2.225087,-96.180016,69.728149
98,3,2,2020.0,-0.394835,-0.534038,-0.425779,0.009920,-0.003727,-0.001900,-44.309849,...,55.229939,-0.137097,-8.235356e-16,2.873583e-16,-0.000449,0.001972,-0.003176,-53.027840,-22.805929,-161.032242


In [5]:
# Id column for y_test
Id = pd.read_csv('Id_bow.csv')
Id.drop(columns=Id.columns[0], axis=1, inplace=True)
Id

,Id
0,0
1,2
2,3
3,4
4,5
...,...
95,103
96,104
97,105
98,106


## Logistic regression with ElasticNet

We apply logistic regression with ElasticNet.

In [6]:
# Following from [5] without cross validation because it takes so much time

elastic_net_classifier = LogisticRegressionCV(cv=None, penalty='elasticnet', solver='saga', l1_ratios=0.1*np.ones(np.shape(X_train)[1]))

In [7]:
# Fit a model to the training data

elastic_net_classifier.fit(X_train, y_train)

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/s

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

...

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/nesli/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:

LogisticRegressionCV(l1_ratios=array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
                     penalty='elasticnet', solver='saga')

In [8]:
y_test=elastic_net_classifier.predict(X_test)

In [9]:
y_test

array([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

In [10]:
y_test=pd.DataFrame(data=y_test, columns=['party'])

In [11]:
y_test

,party
0,1
1,0
2,0
3,0
4,0
...,...
95,1
96,0
97,1
98,1


In [12]:
y_test['party']=y_test['party'].apply(lambda x: 'D' if x==0 else 'R')

In [13]:
y_test.insert(0, "Id", Id['Id'])

In [14]:
y_test

,Id,party
0,0,R
1,2,D
2,3,D
3,4,D
4,5,D
...,...,...
95,103,R
96,104,D
97,105,R
98,106,R


In [15]:
y_test.to_csv('ytest_logreg_istanbul.csv')

## References

[1] https://stackoverflow.com/questions/66787845/how-to-perform-elastic-net-for-a-classification-problem